In [4]:
import os

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatAnthropic
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores.qdrant import Qdrant

from getpass import getpass

In [5]:
os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
os.environ['QDRANT_API_KEY'] = getpass("Enter Qdrant API key:")
qdrant_url = 'https://58de2381-e750-4aed-8eb2-7b08d8faf30b.us-east4-0.gcp.cloud.qdrant.io:6333'
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/RAG-mktg/model_cache/'

Enter Anthropic key: ········
Enter Qdrant API key: ········


In [6]:
# Load the data
loader = CSVLoader(file_path='/mnt/code/data/disease_components.csv',source_column="link")
data = loader.load()

In [7]:
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

1183 1183


In [8]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [11]:
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

doc_store = Qdrant.from_texts(texts,
                          metadatas=metadatas,
                          embedding=embeddings,
                          url=qdrant_url,
                          api_key=os.environ['QDRANT_API_KEY'],
                          collection_name=f"medical_qa_search")

rag_llm = ChatAnthropic(temperature=0,
                        anthropic_api_key=os.environ["ANTHROPIC_API_KEY"])

In [12]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [17]:
user_question = input("Please provide the symptoms here :")
result = qa_chain(user_question)

Please provide the symptoms here : What is the cause of tendinitis?


What is the cause of tendinitis?


In [18]:
result['result']

' Based on the information provided, the main cause of tendinitis is repetitive or intense strain on the tendons over time. Some key points about the causes of tendinitis:\n\n- "Tendinitis is caused by repetitive or intense strain on the tendon, the band of tissue that connects muscle to bone. This tendon is used when you walk, run, jump or push up on your toes. The structure of the tendon weakens with age, which can make it more susceptible to injury — particularly in people who may participate in sports only on the weekends or who have suddenly increased the intensity of their running programs." (Achilles Tendinitis Overview)\n\n- "Tendinitis is an overuse injury of the tendon — the thick fibrous cords that attach muscle to bone. The condition causes pain and tenderness just outside a joint." (Tendinitis Overview) \n\n- "Patellar tendinitis is a common overuse injury, caused by repeated stress on your patellar tendon. The stress results in tiny tears in the tendon, which your body at

In [19]:
result['source_documents'][0].page_content

'name: Tendinitis\nlink: https://www.mayoclinic.org/diseases-conditions/tendinitis/symptoms-causes/syc-20378243\nSymptoms: [\'Signs and symptoms of tendinitis tend to occur at the point where a tendon attaches to a bone and typically include:\', \'Most cases of tendinitis respond to self-care measures. See your doctor if your signs and symptoms persist and interfere with your daily activities for more than a few days.\', \'\', \'\', \'\', \'Pain often described as a dull ache, especially when moving the affected limb or joint\', \'Tenderness\', \'Mild swelling\']\nOverview: [\'\', \'Tendons are thick fibrous cords that attach muscles to bone. Overuse or strain on a joint can irritate tendons and result in tendinitis.\', \'\', \'Tendinitis is inflammation or irritation of a tendon — the thick fibrous cords that attach muscle to bone. The condition causes pain and tenderness just outside a joint.\', "While tendinitis can occur in any of your tendons, it\'s most common around your shoulde